In [ ]:
import pyPhyNR as pynr
from pyPhyNR.core.signal_builder import NRSignalBuilder


In [ ]:
# TDD Configuration
TDD_SLOT_PATTERN = [0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SPECIAL_SLOT_SYMBOL_PATTERN = [0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 1, 1, 1, 1]

print("TDD Slot Pattern (20 slots):")
print(f"Pattern: {TDD_SLOT_PATTERN}")
print(f"DL slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 0]}")
print(f"UL slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 1]}")
print(f"Special slots: {[i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 2]}")

print("\nSpecial Slot Symbol Pattern (14 symbols):")
print(f"Pattern: {SPECIAL_SLOT_SYMBOL_PATTERN}")
print(f"DL symbols: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 0]}")
print(f"UL symbols: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 1]}")
print(f"Guard period: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 2]}")
print(f"Switch point: {[i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 3]}")


In [ ]:
# Create signal builder for 20MHz, numerology 1 (30kHz SCS)
signal = NRSignalBuilder(bandwidth_mhz=20, numerology=1, cell_id=1)
signal.configure_carrier(sample_rate=23.04e6, fft_size=768).initialize_grid()
print(signal.get_carrier_config())


In [ ]:
# Add PDSCH to UL slots (inverted waveform)
ul_slots = [i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 1]
signal.add_pdsch(
    start_rb=0,
    num_rb=51,
    start_symbol=0,
    num_symbols=14,
    slot_pattern=ul_slots,
    modulation="QPSK",
    power=0.0
).add_dmrs(
    dmrs_positions=[2, 11],
    clear_full_symbol=False,
    subcarrier_pattern="even",
    power_offset_db=0.0
)


In [ ]:
# Add PDSCH to UL symbols in special slot
special_slots = [i for i, slot in enumerate(TDD_SLOT_PATTERN) if slot == 2]
ul_symbols_in_special = [i for i, sym in enumerate(SPECIAL_SLOT_SYMBOL_PATTERN) if sym == 1]

if special_slots and ul_symbols_in_special:
    signal.add_pdsch(
        start_rb=0,
        num_rb=51,
        start_symbol=ul_symbols_in_special[0],
        num_symbols=len(ul_symbols_in_special),
        slot_pattern=special_slots,
        modulation="QPSK",
        power=0.0
    ).add_dmrs(
        dmrs_positions=[11],
        clear_full_symbol=False,
        subcarrier_pattern="even",
        power_offset_db=0.0
    )


In [ ]:
# Generate waveform
pynr.utils.plot_grid_dl(signal.carrier_config, signal.grid)
iq_samples = signal.generate_signal(target_rms=2914)
pynr.utils.plot_time_domain(iq_samples, signal.carrier_config)
pynr.utils.plot_frequency_domain(iq_samples, signal.carrier_config)
